# MergeSort Benchmark Injection Testing

In [94]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
%run "../../Setup_Scripts/Setup_Generic.ipynb"
%matplotlib widget
scope.adc.samples = 800

INFO: Found ChipWhisperer😍


In [101]:
scope.dis()
target.dis()

In [2]:
#import necessary library 
import linear_comp as lc
import matplotlib.pyplot as plt 
import chipwhisperer as cw
import numpy as np

In [3]:
def cap_pass_trace():
    #reset_target(scope)
    scope.arm()
    ret = scope.capture()
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    trace = scope.get_last_trace()
    return trace


<h3> This is for Research Benchmark Checking </h3>

In [106]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/Mergesort
gmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
gmake[1]: '.dep' is up to date.


SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (Homebrew AVR GCC 9.4.0) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     .././hal/xmega/XMEGA_AES_driver.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/xmega_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/usart_driver.c ...
-e Done!
.
Compiling:
-en     merge_sort.c ...
-e Done!
.
LINKING:
-en     merge_sort-CWLITEXMEGA.elf ...
-e Done!
.
Creating load file for Flash: merge_sort-CWLITEXMEGA.hex
avr-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature merge_sort-CWLITEXMEGA.elf merge_sort-CWLITEXMEGA.hex
.
Creating load file for Flash: merge_sort-CWLITEXMEGA.bin
avr-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signa

In [107]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/Mergesort/merge_sort-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2479 bytes


In [101]:
#this is the method bloc get all the possible traits from the testing, repeatedly. 
reset_target(scope)
trace_set=[]
for i in range(140):
    trace_hit = cap_pass_trace()
    if(len(trace_set))==0:
        trace_set.append(trace_hit)
    else:
        flag=False
        for j in range(len(trace_set)):
            if(lc.linear_comp(trace_hit,trace_set[j],0.1,80))>750:
                flag=True
        
        if not flag:
            trace_set.append(trace_hit)
    
print(len(trace_set))

1


In [ ]:
cw.plot(trace_set[0])
#lc.linear_comp(trace_set[1],trace_set[2],0.1,40)

#reset_target(scope)
#trace_check = cap_pass_trace()
#cw.plot(trace_check) *cw.plot(trace_set[3]) 
#lc.linear_comp(trace_check,trace_set[0],0.06,50)

# The Testing Part

In [108]:
result=[]
reset_target(scope)

for i in range(1024): # we now are getting right result
    trace_test=cap_pass_trace()
    x=max([lc.linear_comp(trace_test,trace_set[j],0.1,80) for j in range(len(trace_set))])
    result.append(x)

In [109]:
with open("Merge_sort_Accuracy.csv",'w') as f:
    f.write("Test Iteration,Matching,Decision\n")
    for i in range(len(result)):
        if(result[i]>750):
            f.write(f"{i},{result[i]},OK\n")
        else:
            f.write(f"{i},{result[i]},Failed\n")